In [11]:
pip install pandas pyproj

   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.2 MB 1.5 MB/s eta 0:00:04
   ---------- ----------------------------- 1.6/6.2 MB 2.5 MB/s eta 0:00:02
   --------------- ------------------------ 2.4/6.2 MB 2.9 MB/s eta 0:00:02
   ----------------------- ---------------- 3.7/6.2 MB 3.6 MB/s eta 0:00:01
   ------------------------------- -------- 5.0/6.2 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
print("Current Working Directory:", os.getcwd())

pd.read_csv("C:/iea49/WAVES/library/base_2022/project/plant/base_floating_2022_layout.csv")

Current Working Directory: C:\iea49\WAVES\library\base_2022\project\plant


,id,substation_id,name,type,latitude,longitude,string_actual,order_actual,string,order,distance,subassembly,upstream_cable,easting,northing,floris_x,floris_y,cable_length,bury_speed
0,A7,OSS,A7,turbine,-41.705606,-74.677505,0.0,0.0,0.0,0.0,0.0,15MW_core_wind_floating.yaml,array_core_wind.yaml,12822.70,1677.22,12822.70,1677.22,0,0
1,B7,OSS,B7,turbine,-41.662691,-74.670311,0.0,1.0,0.0,1.0,0.0,15MW_core_wind_floating.yaml,array_core_wind.yaml,13404.70,3276.20,13404.70,3276.20,0,0
2,C7,OSS,C7,turbine,-41.619816,-74.663110,0.0,2.0,0.0,2.0,0.0,15MW_core_wind_floating.yaml,array_core_wind.yaml,13986.70,4875.18,13986.70,4875.18,0,0
3,D7,OSS,D7,turbine,-41.576983,-74.655900,0.0,3.0,0.0,3.0,0.0,15MW_core_wind_floating.yaml,array_core_wind.yaml,11388.40,2199.28,11388.40,2199.28,0,0
4,E7,OSS,E7,turbine,-41.534190,-74.648682,0.0,4.0,0.0,4.0,0.0,15MW_core_wind_floating.yaml,array_core_wind.yaml,11970.40,3798.26,11970.40,3798.26,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15MW_core_wind_floating.yaml,array_core_wind.yaml,4257.86,13848.60,4257.86,13848.60,0,0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15MW_core_wind_floating.yaml,array_core_wind.yaml,1659.55,11172.70,1659.55,11172.70,0,0
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15MW_core_wind_floating.yaml,array_core_wind.yaml,2241.53,12771.70,2241.53,12771.70,0,0
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15MW_core_wind_floating.yaml,array_core_wind.yaml,2823.51,14370.70,2823.51,14370.70,0,0


In [32]:
from pathlib import Path

import pandas as pd
from pyproj import CRS
from pyproj.transformer import Transformer

def main(reference_point: tuple[float, float], file_name: Path) -> None:
    
    # Build the transformers and get the reference point
    crs = CRS.from_epsg(32711)
    proj = Transformer.from_crs(crs.geodetic_crs, crs)
    proj_reverse = Transformer.from_crs(crs, crs.geodetic_crs)
    
    base_easting, base_northing = proj.transform(*reference_point)
    
    # Load in the data
    df = pd.read_csv("base_floating_2022_layout.csv")

    spatial_coordinates = df[["easting", "northing"]].values + (base_easting + base_northing)
    coordinates = [proj_reverse.transform(e, n) for (e, n) in spatial_coordinates]

    df[["longitude", "latitude"]] = coordinates
    df.to_csv(file_name.with_stem(f"{file_name.stem}_converted_coordinates"), index=False)


if __name__ == "__main__":
    reference_point = (-124.708, 40.928)  # (lon, lat) or (x, y) in WGS-84
    file_name  = Path("base_floating_2022_layout.csv").resolve()
    
    main(reference_point, file_name)

In [33]:
from pathlib import Path
import pandas as pd
from pyproj import CRS
from pyproj.transformer import Transformer

def main(reference_point: tuple[float, float], file_name: Path) -> None:
    # Build the transformers and get the reference point
    crs_wgs84 = CRS.from_epsg(4326)  # WGS 84
    crs_utm = CRS.from_epsg(32710)   # UTM zone 10S
    proj = Transformer.from_crs(crs_wgs84, crs_utm)
    proj_reverse = Transformer.from_crs(crs_utm, crs_wgs84)
    
    base_easting, base_northing = proj.transform(reference_point[1], reference_point[0])  # (lat, lon)

    # Load in the data
    df = pd.read_csv(file_name)

    spatial_coordinates = df[["easting", "northing"]].values
    coordinates = [proj_reverse.transform(e, n) for (e, n) in spatial_coordinates]

    df[["longitude", "latitude"]] = coordinates
    df.to_csv(file_name.with_stem(f"{file_name.stem}_converted_coordinates"), index=False)

if __name__ == "__main__":
    reference_point = (-124.708, 40.928)  # (lon, lat)
    file_name  = Path("base_floating_2022_layout.csv").resolve()
    
    main(reference_point, file_name)